In [ ]:
import numpy as np, random, operator, pandas as pd
import matplotlib.pyplot as plt

def create_starting_population(size,Number_of_city):
    #Cria uma população inicial de rotas, recebendo como a entrada o tamanho da população e a quantidade de cidades.
    #Vai gerar rotas aleatórias pra cada indivíduo da população.
    population = []
    for i in range(0,size):
        population.append(create_new_member(Number_of_city))
    return population

def pick_mate(N):
    #Seleciona de modo aleatório um indivíduo da população como um 'parceiro de acasalamento'.
    #Recebe como entrada o número máximo de indivíduos presente na população.
    i=random.randint(0,N)
    return i

def distance(i,j):
    #Calcula qual é a distância entre duas cidades 'i' e 'j' com base nas suas coordenadas (x,y).
    return np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2)

def score_population(population, CityList):
    # Calcula a aptidão (Distância total percorrida) pra cada rota na população.
    # Recebe como entrada a população e uma lista de cidades.
    scores = []
    for i in population:
        scores.append(fitness(i, CityList))
    return scores

def fitness(route,CityList):
    #Calcula a aptidão de uma rota específica.
    #Recebe como entrada uma rota e a lista de cidades e percorre a rota calculando qual a distância total percorrida.
    score=0
    for i in range(1,len(route)):
        k=int(route[i-1])
        l=int(route[i])
        score = score + distance(CityList[k],CityList[l])
    return score

def create_new_member(Number_of_city):
    #Cria uma nova rota aleatória. Recebe como entrada o número de cidades.
    #Gera uma lista com todas as cidades e embaralha essa lista para obter uma rota aleatória.
    pop = list(np.arange(Number_of_city, dtype=int))
    route = random.sample(pop, Number_of_city)
    return route

def crossover(a,b):
    #Realiza o operador de crossover entre duas rotas 'a' e 'b'.
    #Seleciona uma parte da rota a e insere essa parte na mesma posição na rota b. Retorna a nova rota resultante do crossover.
    child=[]
    childA=[]
    childB=[]
    geneA=int(random.random()* len(a))
    geneB=int(random.random()* len(a))
    start_gene=min(geneA,geneB)
    end_gene=max(geneA,geneB)
    for i in range(start_gene,end_gene):
        childA.append(a[i])
    childB=[item for item in a if item not in childA]
    child=childA+childB
    return child

def mutate(route,probablity):
    #Realiza a mutação em uma rota, com uma determinada probabilidade para cada posição da rota.
    #Para cada posição, a rota pode ser trocada com outra posição aleatória.
    route=np.array(route)
    for swaping_p in range(len(route)):
        if(random.random() < probablity):
            swapedWith = np.random.randint(0,len(route))
            temp1=route[swaping_p]
            temp2=route[swapedWith]
            route[swapedWith]=temp1
            route[swaping_p]=temp2
    return route

def selection(popRanked, eliteSize):
    #Seleciona os melhores indivíduos (rotas) da população atual com base em seus rankings.
    #Retorna uma lista com os índices dos indivíduos selecionados.
    selectionResults=[]
    result=[]
    for i in popRanked:
        result.append(i[0])
    for i in range(0,eliteSize):
        selectionResults.append(result[i])
    return selectionResults

def rankRoutes(population,City_List):
    #Calcula o ranking de todas as rotas da população com base em sua aptidão (distância percorrida).
    #Retorna uma lista de tuplas contendo o índice da rota e sua aptidão, ordenadas pelo ranking.
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = fitness(population[i],City_List)
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = False)

def breedPopulation(mating_pool):
    #Realiza o cruzamento entre os indivíduos da população de acasalamento (mating_pool).
    #Gera novos indivíduos através do operador de crossover.
    children=[]
    for i in range(len(mating_pool)-1):
            children.append(crossover(mating_pool[i],mating_pool[i+1]))
    return children

def mutatePopulation(children,mutation_rate):
    # Realiza a mutação em uma população de indivíduos (rotas) com uma determinada taxa de mutação.
    #Aplica a função mutate em cada indivíduo da população.
    new_generation=[]
    for i in children:
        muated_child=mutate(i,mutation_rate)
        new_generation.append(muated_child)
    return new_generation

def matingPool(population, selectionResults):
    #Cria a população de acasalamento (matingpool) com base nos resultados da seleção.
    #Seleciona os indivíduos da população com base nos índices fornecidos em selectionResults.
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def next_generation(City_List,current_population,mutation_rate,elite_size):
    #Gera a próxima geração de indivíduos.
    #Calcula o ranking da população atual, realiza a seleção dos melhores indivíduos, cria a população de acasalamento, realiza o cruzamento e a mutação para gerar a próxima geração.
    population_rank=rankRoutes(current_population,City_List)
    selection_result=selection(population_rank,elite_size)
    mating_pool=matingPool(current_population,selection_result)
    children=breedPopulation(mating_pool)
    next_generation=mutatePopulation(children,mutation_rate)
    return next_generation

def genetic_algorithm(City_List,size_population=1000,elite_size=75,mutation_Rate=0.01,generation=2000):
    #Implementa o algoritmo genético completo.
    #Recebe como entrada a lista de cidades, o tamanho da população, o tamanho da elite, a taxa de mutação e o número de gerações.
    #Gera a população inicial, itera através das gerações, realiza a seleção, o cruzamento e a mutação para gerar a próxima geração, e retorna a melhor rota encontrada.


    pop=[]
    progress = []

    Number_of_cities=len(City_List)
    population=create_starting_population(size_population,Number_of_cities)
    progress.append(rankRoutes(population,City_List)[0][1])
    print(f"Distância da Rota Inicial: {progress[0]}")
    print(f"Rota Inicial: {population[0]}")
    for i in range(0,generation):
        pop = next_generation(City_List,population,mutation_Rate,elite_size)
        progress.append(rankRoutes(pop,City_List)[0][1])
    rank_=rankRoutes(pop,City_List)[0]
    print(f"Melhor Rota :{pop[rank_[0]]} ")
    print(f"Distância da melhor rota : {rank_[1]}")
    plt.plot(progress)
    plt.ylabel('Distância')
    plt.xlabel('Geração')
    plt.show()

    return rank_, pop

cityList = []


for i in range(0,15):
    x=int(random.random() * 200)
    y=int(random.random() * 200)
    cityList.append((x,y))

rank_,pop=genetic_algorithm(City_List=cityList)

x_axis=[]
y_axis=[]
for i in cityList:
    x_axis.append(i[0])
    y_axis.append(i[1])


plt.scatter(x_axis,y_axis)
plt.show()